In [1]:
#include <iostream>
#include <thread>

In [2]:
namespace n1 {
    void f(int i, std::string const& s)
    {
        // just for delay
        int sum = 0;
        for (int i = 0; i < 1000000; ++i) {
            sum += s.length();
        }

        std::cout << i << ", " << s << ", " << sum << "\n";
    }
    std::thread t(f, 3, "hello");
}

3, hello, 5000000


In [3]:
{
    using namespace n1;
    int some_param = 10;

    char buffer[64]; // 1
    sprintf(buffer, "%i", some_param);

    std::thread t(f, 3, buffer); // 2
    t.detach();
    
    // 运行到这里，函数返回，buffer立刻被回收，而thread可能没有结束，所以buffer里面可能是任何值
    std::thread t2(f, 3, std::string(buffer)); // 解决方法，使用std::string，避免悬垂指针
    t2.detach();
}

3, 10, 2000000
3, 10, 2000000


还可能遇到相反的情况：期望传递一个引用，但整个对象被复制了。当线程更新一个引用传递的数据结构时，这种情况就可能发生，比如：